## 2d. Evidence - Resource Utilization QAS Measurements

Now we collect stored, CPU and memory usage data when predicting with the model, for the Performance scenario.

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from demo.scenarios.session import *

### Measurements

Prepare and execute measurements.

In [ ]:
from mlte.measurement.storage import LocalObjectSize
from mlte.evidence.types.real import Real
from mlte.measurement.units import Units

store_measurement = LocalObjectSize("model size")
size: Real = store_measurement.evaluate(MODELS_DIR, unit=Units.byte)
print(size)
size.save(force=True)

In [ ]:
from mlte.measurement.process_measurement import ProcessMeasurement
from mlte.measurement.cpu import LocalProcessCPUUtilization, CPUStatistics

cpu_measurement = LocalProcessCPUUtilization("predicting cpu")
cpu_stats: CPUStatistics = cpu_measurement.evaluate(MODEL_COMMAND)
print(cpu_stats)
cpu_stats.save(force=True)

In [ ]:
from mlte.measurement.memory import (
    LocalProcessMemoryConsumption,
    MemoryStatistics,
)

mem_measurement = LocalProcessMemoryConsumption("predicting memory")
mem_stats: MemoryStatistics = mem_measurement.evaluate(MODEL_COMMAND)
print(mem_stats)
mem_stats.save(force=True)

We can also avoid starting the training process twice by using the asynch methods for both measurements. We start the training process once and pass the id to both measurements.

In [ ]:
from mlte.measurement.process_measurement import ProcessMeasurement
from mlte.measurement.cpu import LocalProcessCPUUtilization, CPUStatistics
from mlte.measurement.memory import (
    LocalProcessMemoryConsumption,
    MemoryStatistics,
)

# Create measurements
cpu_measurement = LocalProcessCPUUtilization("predicting cpu")
mem_measurement = LocalProcessMemoryConsumption("predicting memory")

# Start the process to measure.
pid = ProcessMeasurement.start_script(MODEL_COMMAND[1], MODEL_COMMAND[2:])

# Execute the measurements
cpu_measurement.evaluate_async(pid)
mem_measurement.evaluate_async(pid)
cpu_stats: CPUStatistics = cpu_measurement.wait_for_output()
mem_stats: MemoryStatistics = mem_measurement.wait_for_output()

# Inspect values
print(cpu_stats)
print(mem_stats)

# Save to artifact store
cpu_stats.save(force=True)
mem_stats.save(force=True)